In [1]:
from sqlalchemy import Column, Integer, String, create_engine, ForeignKey
from sqlalchemy.ext.declarative import declarative_base

In [2]:
from sqlalchemy.orm import sessionmaker

In [3]:
engine = create_engine('sqlite:///corpus.db', echo=True)

In [4]:
base = declarative_base()

In [5]:
class SejongCorpus(base):
    __tablename__ = 'SEJONG_CORPUS'
    
    pk = Column('pk', Integer, primary_key=True, autoincrement=True)
    ko_text = Column('ko_text', String, nullable=True)
    thai_text = Column('thai_text', String, nullable=True)
    file_name = Column('file_name', String, nullable=True)
    sent_count = Column('sent_count', Integer, nullable=True)
    cumsum_count = Column('cumsum_count', Integer, nullable=True)
    
    def __repr__(self):
        return 'pk:{}, ko_text:{}, thai_text:{}'.format(self.pk, self.ko_text, self.thai_text)

In [6]:
Session = sessionmaker(bind=engine)
session = Session()

In [51]:
unicode = [r[0] for r in session.query(SejongCorpus.ko_text, SejongCorpus.thai_text).all() if re.match(".*[\u2e80-\u2eff\u31c0-\u31ef\u3200-\u32ff\u3400-\u4dbf\u4e00-\u9fbf\uf900-\ufaff].*", r[0]) is not None]

2020-08-18 23:03:00,963 INFO sqlalchemy.engine.base.Engine SELECT "SEJONG_CORPUS".ko_text AS "SEJONG_CORPUS_ko_text", "SEJONG_CORPUS".thai_text AS "SEJONG_CORPUS_thai_text" 
FROM "SEJONG_CORPUS"
2020-08-18 23:03:00,965 INFO sqlalchemy.engine.base.Engine ()


In [46]:
ko_corpus = [r[0] for r in session.query(SejongCorpus.ko_text, SejongCorpus.thai_text).all() if re.match('.*[가-힣a-zA-Z0-9].*', r[0]) is not None]

2020-08-18 23:00:07,717 INFO sqlalchemy.engine.base.Engine SELECT "SEJONG_CORPUS".ko_text AS "SEJONG_CORPUS_ko_text", "SEJONG_CORPUS".thai_text AS "SEJONG_CORPUS_thai_text" 
FROM "SEJONG_CORPUS"
2020-08-18 23:00:07,744 INFO sqlalchemy.engine.base.Engine ()


In [48]:
ko_corpus

['아이티의 임시 정부를 이끌게 될 대통령으로 13일 취임한 에르타 트루이요 여사는 남성 우위의 나라에서 수많은 여성 최초의 기록을 세운 강인한 인상의 존경받는 판사이다.',
 '트루이요는 34세의 나이에 대법원 12 명의 법관 중 최초의 여성 법관으로 임명됐으며 아이티의 법조 협회 첫 여성 회원이다.',
 '아이티 여성의 법적 지위등 2권의 책을 저술한 트루이요는 임시 대통령직을 수락하면서 아이티 여성을 위해 봉사하겠다고 말할 정도의 여권 운동가이기도 하다.',
 '그녀가 지금부터 해 내야 할 일은 지난 86년 뒤발리에의 독재 정권이 무너진 지 4년 동안 세 차례의 쿠데타와 다섯 번의 정권 교체로 얼룩진 아이티에 자유 총선을 실시하는 일이다.',
 '이것은 너무나도 벅찬 임무라서 누구도 감히 떠맡으려하지 않았던 일이다.',
 '트루이요에 앞서 4명의 대법관이 대통령직 수락을 요청받았으나 모두 고사했던 것으로 알려지고 있다.',
 '처녀 적 이름이 에르타 파스칼인 트루이요는 46년 8월 21일 지방 도시 고나이브에서 9남매 중 막내로 태어났다.',
 '프르토프랭스의 대학에 진학한 트루이요는 은사이자 판사인 20살 연상의 에르네스트 트루이요를 만나 결혼하게 된다.',
 '남편 트루이요의 도움으로 법학 공부를 계속한 그녀는 71년 고나이브대학원을 졸업하면서 동시에 법조 협회의 회원 자격을 얻었다.',
 '나중에 판사직에 승진한 트루이요는 항소 법원 판사를 거쳐 80년 대법관에 임명됐다.',
 '87년 심장 마비로 사망한 남편이 독재자 뒤발리에와 친분이 두터웠던 점이 트루이요의 고속 승진을 가능케 한 것으로 분석되고 있으나 트루이요 여사 자신은 뒤발리에 편이란 소리를 듣지는 않았다.',
 '트루이요 여사는 14살짜리 딸 하나를 두고 있다.',
 '흔히 조평으로 불리는 이 단체는 남북 대화 및 통일 관계 대남 선전을 펴는 노동당 전위 조직들 중 하나다.',
 '조평은 4.19직후 남한에서 제기됐던 남북 협상론 등에 호응하기 위해 61년 홍명희를 위원장으로 급조됐다.'

In [54]:
ko_corpus = [r[0] for r in session.query(SejongCorpus.ko_text, SejongCorpus.thai_text).all() if re.match(".*[\u2e80-\u2eff\u31c0-\u31ef\u3200-\u32ff\u3400-\u4dbf\u4e00-\u9fbf\uf900-\ufaff].*", r[0]) is None]
thai_corpus = [r[1] for r in session.query(SejongCorpus.ko_text, SejongCorpus.thai_text).all() if re.match(".*[\u2e80-\u2eff\u31c0-\u31ef\u3200-\u32ff\u3400-\u4dbf\u4e00-\u9fbf\uf900-\ufaff].*", r[0]) is None]

2020-08-18 23:04:21,913 INFO sqlalchemy.engine.base.Engine SELECT "SEJONG_CORPUS".ko_text AS "SEJONG_CORPUS_ko_text", "SEJONG_CORPUS".thai_text AS "SEJONG_CORPUS_thai_text" 
FROM "SEJONG_CORPUS"
2020-08-18 23:04:21,921 INFO sqlalchemy.engine.base.Engine ()
2020-08-18 23:04:32,979 INFO sqlalchemy.engine.base.Engine SELECT "SEJONG_CORPUS".ko_text AS "SEJONG_CORPUS_ko_text", "SEJONG_CORPUS".thai_text AS "SEJONG_CORPUS_thai_text" 
FROM "SEJONG_CORPUS"
2020-08-18 23:04:32,980 INFO sqlalchemy.engine.base.Engine ()


In [55]:
len(ko_corpus), len(thai_corpus)

(477804, 477804)

In [56]:
from sklearn.model_selection import train_test_split

In [57]:
corpus_train_ko, corpus_test_ko = train_test_split(ko_corpus, test_size=0.2) # train / test 
corpus_train_ko, corpus_valid_ko = train_test_split(corpus_train_ko, test_size=0.1) # train / validation
corpus_train_th, corpus_test_th = train_test_split(thai_corpus, test_size=0.2) # train /test 
corpus_train_th, corpus_valid_th = train_test_split(corpus_train_th, test_size=0.1) # train / validation

In [58]:
len(corpus_train_ko), len(corpus_valid_ko), len(corpus_test_ko)

(344018, 38225, 95561)

In [59]:
with open('corpus.train.ko', 'w+', encoding='utf-8') as f:
    for ko in corpus_train_ko:
        f.write(ko+'|')
        
with open('corpus.valid.ko', 'w+', encoding='utf-8') as f:
    for ko in corpus_valid_ko:
        f.write(ko+'|')
        
with open('corpus.test.ko', 'w+', encoding='utf-8') as f:
    for ko in corpus_test_ko:
        f.write(ko+'|')
        
with open('corpus.train.th', 'w+', encoding='utf-8') as f:
    for th in corpus_train_th:
        f.write(th+'|')
        
with open('corpus.valid.th', 'w+', encoding='utf-8') as f:
    for th in corpus_valid_th:
        f.write(th+'|')
        
with open('corpus.test.th', 'w+', encoding='utf-8') as f:
    for th in corpus_test_th:
        f.write(th+'|')

In [30]:
corpus_all = session.query(SejongCorpus.ko_text, SejongCorpus.thai_text).all()

2020-08-10 12:24:09,760 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-08-10 12:24:09,779 INFO sqlalchemy.engine.base.Engine ()
2020-08-10 12:24:09,786 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-08-10 12:24:09,788 INFO sqlalchemy.engine.base.Engine ()
2020-08-10 12:24:09,792 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-10 12:24:09,806 INFO sqlalchemy.engine.base.Engine SELECT "SEJONG_CORPUS".ko_text AS "SEJONG_CORPUS_ko_text", "SEJONG_CORPUS".thai_text AS "SEJONG_CORPUS_thai_text" 
FROM "SEJONG_CORPUS"
2020-08-10 12:24:09,810 INFO sqlalchemy.engine.base.Engine ()


In [31]:
with open('sejong.txt', 'w+', encoding='utf-8') as f:
    for corpus in corpus_all:
        f.write(corpus[0]+'\t'+corpus[1]+'\n')

In [32]:
with open('sejong.txt', 'r', encoding='utf-8') as f:
    a = f.read()

In [33]:
a_list = a.split('\n')

In [34]:
a_list[0]

'아이티의 임시 정부를 이끌게 될 대통령으로 13일 취임한 에르타 트루이요 여사는 남성 우위의 나라에서 수많은 여성 최초의 기록을 세운 강인한 인상의 존경받는 판사이다.\tนางเออร์ตาทรูโยซึ่งเข้ารับตำแหน่งเมื่อวันที่ 13 ในฐานะประธานาธิบดีซึ่งจะเป็นผู้นำรัฐบาลชั่วคราวของเฮติเป็นผู้พิพากษาที่ได้รับความเคารพและมีความประทับใจอย่างมากซึ่งสร้างสถิติเป็นครั้งแรกสำหรับผู้หญิงจำนวนมากในประเทศที่มีการปกครองแบบผู้ชาย'